In [2]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io
from skimage.registration import phase_cross_correlation
from collections import defaultdict
from tqdm.notebook import trange, tqdm, tqdm_notebook
from joblib import Parallel, delayed
import re
import pickle
from matplotlib.colors import ListedColormap

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# Import path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

from config import *

In [5]:
csv_file = data_meta / "info.csv"
csv_exist = csv_file.is_file()

# Create directory if not existL
try:
    data_processed.mkdir(parents=True, exist_ok=False)
except FileExistsError:
    print("Processed image folder is already there")
else:
    print("Processed image folder was created")

if csv_exist:
    print("Info csv file already exists")
else:
    print("No Info csv file")

Processed image folder is already there
Info csv file already exists


# Get folder information

Read information to pandas dataframe of the image folder

In [6]:
channel2marker = {
    "Cycle1": {
        "CH1": "Hoeschst", 
        "CH2": "p-EGFR"},
    "Cycle2": {
        "CH1": "Hoeschst", 
        "CH3": "Phalloidin", 
        "CH4": "WGA"},
    "Cycle3": {
        "CH1": "Hoeschst",
        "CH2": "Non-p-B-catenin",
        "CH3": "APC",
        "CH4": "DKK1"},
    "Cycle4": {
        "CH1": "Hoeschst", 
        "CH2": "Cyclin E", 
        "CH3": "EMMPRIN",
        "CH4": "Wnt1"},
    "Cycle5": {
        "CH1": "Hoeschst", 
        "CH2": "p-AKT",
        "CH3": "p-B-catenin",
        "CH4": "RNF 43"},
    "Cycle6": {
        "CH1": "Hoeschst",
        "CH2": "AKT",
        "CH3": "p-mTOR",
        "CH4": "mTOR",
    },
    "Cycle7": {
        "CH1": "Hoeschst",
        "CH2": "DKK2",
        "CH3": "AXIN1",
        "CH4": "EGFR",
    },
    "Cycle8": {
        "CH1": "Hoeschst",
        "CH3": "Cyclin D1",
        "CH4": "B-tubin",
    },
    "Cycle9": {
        "CH1": "Hoeschst",
        "CH2": "H3k27me3-488",
        "CH3": "H3k9Ac-555",
        "CH4": "H3k4me3-647",
    },
    "Cycle10": {
        "CH1": "Hoeschst", 
        "CH2": "Concanavalin A",
        "CH4": "PCNA"
    },
    "Cycle11": {
        "CH1": "Hoeschst", 
        "CH2": "B-actin",
    }
}


def get_info(img_folder, name_dict=channel2marker, save=False):
    """Function returns the info from folder containing multi-cycle staigning on cell

    Args:
        img_folder (str) : imgage folder path to get information
        name_dict (dict) : three level dictionnary mapping cycle -> channel -> marker name

    Returns:
        pandas dataframe with information
    """
    locations = []
    cycles = []
    z_stacks = []
    channels = []
    images_path = []
    markers = []
    after_bleach = []

    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(img_folder):
        for name in sorted(filenames):

            if "tif" in name and "Overlay" not in name and "stitched" not in name and 'Composite' not in name:
                # Get information from image name

                z_stack = name.split("_")[2][1:]
                location = name.split("_")[1]
                channel = name.split("_")[-1].split(".")[0]
                cycle = dirpath.split("\\")[-1].split("_")[1]

                if z_stack == "":
                    continue

                # skip if no markers in image
                try:
                    marker = name_dict[cycle][channel]
                except:
                    continue


                if "AftBleach" in dirpath:
                    after_bleach.append(True)
                else:
                    after_bleach.append(False)

                cycles.append(cycle)
                locations.append(int(location))
                z_stacks.append(int(z_stack[1:]))
                channels.append(channel)
                markers.append(marker)
                images_path.append(os.path.join(dirpath, name))
    info = {
        "Cycle": cycles,
        "Location": locations,
        "After_bleach": after_bleach,
        "Z_stack": z_stacks,
        "Channel": channels,
        "Marker": markers,
        "Path": images_path,
    }
    df = pd.DataFrame(info)

    if save == True:
        df.to_csv(data_meta / "info.csv", index=False)

    df.Z_stack = pd.to_numeric(df.Z_stack)

    df["Path_corrected"] = ""

    return df

In [7]:
if not csv_exist:
    df = get_info(data_raw, save=True)
    print("Created pandas dataframe")
else:
    df = pd.read_csv(csv_file)
    print("Imported pandas dataframe")

Imported pandas dataframe


In [8]:
df.head()

,Cycle,Location,After_bleach,Z_stack,Channel,Marker,Path
0,Cycle10,1,False,1,CH1,Hoeschst,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
1,Cycle10,1,False,1,CH2,Concanavalin A,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
2,Cycle10,1,False,1,CH4,PCNA,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
3,Cycle10,1,False,2,CH1,Hoeschst,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
4,Cycle10,1,False,2,CH2,Concanavalin A,Y:\coskun-lab\Thomas\Spatial signaling\dataset...


# Image background subtraction

Image shadding correction using background subtration of gaussian blurr

In [9]:
def extract_condition_channel(df, cycle, condition, channel):
    """Function returns dataframe of specified channel image based on condition from input dataframe

    Args:
        df (pd DataFrame) : info dataframe for all images
        condition (str) : condition name

    Returns:
        pandas dataframe with information
    """
    return df[
        (df["Location"] == condition)
        & (df["Channel"] == channel)
        & (df["Cycle"] == cycle)
    ]


def background_correction(
    df, filtersize=1025, sigma=256, folder=data_processed, save=False, show=False
):
    """Function to perform background substraction for image using gaussian blurr of original image

    Args:
        df (pd DataFrame) : info dataframe for all images
        filtersize (int) : filter size of gaussian kernel
        sigma (int) : sigma of guassian blurr
        folder (str) : folder to save corrected images
        save (bool) : bool to save the image
        show (bool) : bool to show corrected image and gaussian blur of original image

    Returns:
        None
    """
    # Loop through the rows of the dataframe
    for row in df.itertuples():
        # Define saving filename for corrected image
        if row.After_bleach == False:
            filename = "_".join([str(row.Location), str(row.Z_stack), row.Cycle, row.Marker])
        else:
            filename = "_".join(['Af', str(row.Location), str(row.Z_stack), row.Cycle, row.Marker])
        path = os.path.join(folder, filename + ".tiff")


        # Save image
        if save:
            df.at[row.Index, "Path_corrected"] = path

In [10]:
csv_file = data_meta / "all_info.csv"

if not csv_file.is_file():
    background_correction(df, save=True)
    df.to_csv(csv_file, index=False)
    print("Created images with background correction")
else:
    df = pd.read_csv(csv_file)

# Find best focus

In [11]:
from skimage import exposure

def blur_score(img):
    fm = cv2.Laplacian(img, cv2.CV_64F).var()
    return fm

def detect_blur_fft(image, size=60):
    # grab the dimensions of the image and use the dimensions to
    # derive the center (x, y)-coordinates
    (h, w) = image.shape
    (cX, cY) = (int(w / 2.0), int(h / 2.0))
    
    # compute the FFT to find the frequency transform, then shift
    # the zero frequency component (i.e., DC component located at
    # the top-left corner) to the center where it will be more
    # easy to analyze
    fft = np.fft.fft2(image)
    fftShift = np.fft.fftshift(fft)
    
    # zero-out the center of the FFT shift (i.e., remove low
    # frequencies), apply the inverse shift such that the DC
    # component once again becomes the top-left, and then apply
    # the inverse FFT
    fftShift[cY - size:cY + size, cX - size:cX + size] = 0
    fftShift = np.fft.ifftshift(fftShift)
    recon = np.fft.ifft2(fftShift)
    
    magnitude = 20 * np.log(np.abs(recon))
    mean = np.mean(magnitude)
    
    return mean

def read_img(path):
    return skimage.io.imread(path)

def contrast_streching(img):
    p2, p98 = np.percentile(img, (0.5, 99.5))
    return exposure.rescale_intensity(img, in_range=(p2, p98))

def joblib_loop(task, pics):
    return Parallel(n_jobs=20)(delayed(task)(i) for i in pics)

def get_blur_example(blur_scores, imgs, path):
    blur_max = np.argmax(blur_scores)
    blur_min = np.argmin(blur_scores)
    
    fig = plt.figure(figsize=(10,7))
    gs = fig.add_gridspec(nrows=3, ncols=2, wspace=0.02, hspace=0.02)
    ax0 = fig.add_subplot(gs[:2,0])
    ax0.imshow(contrast_streching(imgs[blur_min]), cmap='gray')
    ax0.axis('off')
    ax0.title.set_text(f'{blur_min} - {blur_scores[blur_min]}')
    
    ax1 = fig.add_subplot(gs[:2,1])
    ax1.imshow(contrast_streching(imgs[blur_max]), cmap='gray')
    ax1.axis('off')
    ax1.title.set_text(f'{blur_max} - {blur_scores[blur_max]}')
    
    ax2 = fig.add_subplot(gs[2,:])
    ax2.plot(blur_scores)
    ax2.set_xlabel('Z stack')
    ax2.set_ylabel('Blur score')
    
    fig.savefig(path, bbox_inches='tight')
    plt.close(fig)
    return blur_max


In [12]:
csv_blur = data_meta / "blur.csv"

df = df[df.Location < 64]
df = df[df.Channel == 'CH1']

if not csv_blur.is_file():
    grouped = df.groupby(['Location', 'Cycle', 'After_bleach'])
    max_blur_values = []
    for name, group in tqdm(grouped):
        paths = group.Path_corrected.tolist()
        imgs = joblib_loop(read_img, paths)
        blur_scores = joblib_loop(detect_blur_fft, imgs)
        savepath = figure_dir / 'blur' / f'{name[0]}_{name[1]}_{name[2]}.tiff' 
        blur_max = get_blur_example(blur_scores, imgs, savepath)
        values = list(name)
        values.append(blur_max)
        max_blur_values.append(values)

    df_blur_max = pd.DataFrame(max_blur_values, columns=['Location', 'Cycle', 'After_bleach', 'Z_stack_focus'])
    df_blur_max.to_csv(data_meta / "blur.csv", index=False)
else:
    df_blur_max = pd.read_csv(csv_blur)

In [13]:
df_blur_max

,Location,Cycle,After_bleach,Z_stack_focus
0,1,Cycle1,False,11
1,1,Cycle1,True,11
2,1,Cycle10,False,17
3,1,Cycle10,True,16
4,1,Cycle11,False,14
...,...,...,...,...
1318,63,Cycle7,True,15
1319,63,Cycle8,False,15
1320,63,Cycle8,True,18
1321,63,Cycle9,False,20


# Combine best focus

In [17]:
csv_file = data_meta / "all_info.csv"

df = pd.read_csv(csv_file)
df = df[df.Location < 64]

In [55]:
def get_maximum_projection(df, img_folder, n=2, save=False):
 
    df_combined = pd.DataFrame(columns=['Cycle', 'Location', 'Channel', 'Marker', 'Path'])
    df_grouped = df.groupby(['Location', 'Cycle', 'Marker', 'After_bleach'])
    # Loop through condition and cycle pair
    for name, df_subset in tqdm(df_grouped):
        df_blur = df_blur_max.query('Location==@name[0] & Cycle==@name[1] & After_bleach==@name[3]')
        z_focus = df_blur.Z_stack_focus.item()
        z_focus_min = z_focus - n
        z_focus_max = z_focus + n

        df_subset_focus = df_subset.query('Z_stack >= @z_focus_min & Z_stack <= @z_focus_max' )
    
        paths = df_subset_focus.Path_corrected.tolist()
        imgs = joblib_loop(read_img, paths)
        
        row = df_subset_focus.iloc[0,:]
        
        # Save image
        img_combined = np.amax(imgs, axis=0, keepdims=True)[0]
        filename = "_".join([str(row.Location), row.Cycle, row.Marker, str(row.After_bleach)])
        path = os.path.join(img_folder, filename + ".tif")
        cv2.imwrite(path, img_combined)

        # Append dataframe
        new_row = {
            "Cycle": row.Cycle,
            "Location": row.Location,
            "Channel": row.Channel,
            "Marker": row.Marker,
            "After_bleach": row.After_bleach,
            "Path": path,
        }

        df_combined = df_combined.append(new_row, ignore_index=True)

    if save:
        df_combined.to_csv(data_meta / 'info_combined.csv', index=False)

In [56]:
info_combined = data_meta / 'info_combined.csv'

if not info_combined.is_file():
    get_maximum_projection(df, data_combined, save=True)

  0%|          | 0/4536 [00:00<?, ?it/s]

# Image registration

Image registration using the DAPI channel across images

In [57]:
df.head()

,Cycle,Location,After_bleach,Z_stack,Channel,Marker,Path,Path_corrected
0,Cycle10,1,False,1,CH1,Hoeschst,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
1,Cycle10,1,False,1,CH2,Concanavalin A,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
2,Cycle10,1,False,1,CH4,PCNA,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
3,Cycle10,1,False,2,CH1,Hoeschst,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,Y:\coskun-lab\Thomas\Spatial signaling\dataset...
4,Cycle10,1,False,2,CH2,Concanavalin A,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,Y:\coskun-lab\Thomas\Spatial signaling\dataset...


In [79]:
def get_shift_within_cycle(df):
    """Function to get shift within each cycle of the DAPI channel

    Args:
        df (pd DataFrame) : info dataframe for images

    Returns:
        print shift values between each level
    """
    # Get elements of DAPI channel
    df = df[df["Marker"] == "Hoeschst"]

    # Loop through location
    locations = df.Location.unique()
    for location in tqdm(locations, desc='location'):
        for cycle in tqdm(df.Cycle.unique(), desc='cycle', leave=False):
            # Read the cycle info and z level of 1 for reference dapi channel
            df_subset = df[(df.Cycle == cycle) & (df.Location == location)]
            for af in df_subset.After_bleach.unique():
                temp = df_subset[df_subset.After_bleach == af]
                reference = temp[temp.Z_stack == 1]
                reference_dapi = skimage.io.imread(reference.Path_corrected.item())

                # number of z level in this cycle
                n_channel = temp.Z_stack.max() + 1
                shift_x = []
                shift_y = []
                # Get shift between z level
                for i in range(2, n_channel):
                    img_dapi = skimage.io.imread(
                        temp[temp.Z_stack == i].Path_corrected.item()
                    )
                    try:
                        shift, error, diffphase = phase_cross_correlation(
                            reference_dapi, img_dapi
                        )
                        shift_x.append(shift[0])
                        shift_y.append(shift[1])
                    except ValueError as err:
                        print(err)
                        print(str(temp[temp.Z_stack == i].Path_corrected.item()))
                        print(str(reference.Path_corrected.item()))
                if af:
                    print(
                        f"For AF {cycle} detected subpixel offset ({np.mean(shift_x)}, {np.mean(shift_y)}) between"\
                        f" for location {location} - {sub_location}"
                    )
                else:
                    print(
                        f"For {cycle} detected subpixel offset ({np.mean(shift_x)}, {np.mean(shift_y)}) between"\
                        f" for location {location} - {sub_location}"
                    )

def get_shift_between_cycle(df):
    """Function to get shift within each cycle for the DAPI channel

    Args:
        df (pd DataFrame) : info dataframe for images in dapi channel accross cycle

    Returns:
        shift dictionnary
    """
    # Get elements of DAPI channel
    df = df[df["Marker"] == "Hoeschst"]

    # Get shift between cycle
    cycles = df.Cycle.unique().tolist()
    cycles.sort(key=lambda f: int(re.sub('\D', '', f)))
    locations = df.Location.unique()
    shift_dict = defaultdict(dict)
    for location in tqdm(locations, desc='location'):
        # Get subset of specific location
        df_subset = df[df.Location == location]

        # Get reference cycle
        temp = df_subset[(df_subset.Cycle == cycles[0]) & (df_subset.After_bleach == False)]
        reference = temp[temp.Z_stack == 1]
        reference_dapi = skimage.io.imread(reference.Path_corrected.item())

        # Define shift dictionnary
        shift_dict[location] = defaultdict(dict)

        # Max shift accross all cycle:
        max_shift_x = 0
        min_shift_x = 0
        max_shift_y = 0
        min_shift_y = 0

        for af in df_subset.After_bleach.unique():
            if af: 
                cycles_it = cycles[:-1] # if after bleach no cycle 8
            else:
                cycles_it = cycles[1:] # if normal cycle 1 as reference

            for cycle in tqdm(cycles_it, desc='cycle', leave=False):
                temp = df_subset[(df_subset.Cycle == cycle) & (df_subset.After_bleach == af)]
                img_dapi = skimage.io.imread(temp[temp.Z_stack == 1].Path_corrected.item())

                     
                # Get image shift y and x and save to shift_dict
                try:
                    shift, _, _ = phase_cross_correlation(
                        reference_dapi, img_dapi
                    )  # Shift vector required to register moving images with reference images. Axis orderingis constitent with Y,X
                except ValueError as err:
                    print(err)
                    print(str(temp[temp.Z_stack == 1].Path_corrected.item()))
                    print(str(reference.Path_corrected.item()))
#                 print(
#                     f"For {cycle} detected subpixel offset ({shift[0]}, {shift[1]}) compared to {cycles[0]} "\
#                     f"for location {location} for AF: {af}"
#                 )
                shift_y, shift_x = shift[0], shift[1]

                if af:
                    shift_dict[location][cycle]['af'] = {"shift_x": shift_x, "shift_y": shift_y}
                else:
                    shift_dict[location][cycle]['norm'] = {"shift_x": shift_x, "shift_y": shift_y}

                # Update max shift
                max_shift_x = shift_x if shift_x > max_shift_x else max_shift_x
                min_shift_x = shift_x if shift_x < min_shift_x else min_shift_x
                max_shift_y = shift_y if shift_y > max_shift_y else max_shift_y
                min_shift_y = shift_y if shift_y < min_shift_y else min_shift_y

        max_shift_x = int(max_shift_x)
        min_shift_x = int(min_shift_x)
        max_shift_y = int(max_shift_y)
        min_shift_y = int(min_shift_y)

        shift_dict[location]["max"] = {
            "max_shift_x": max_shift_x,
            "min_shift_x": min_shift_x,
            "max_shift_y": max_shift_y,
            "min_shift_y": min_shift_y,
        }

    return shift_dict

def shift_crop(df, shift_dict, save=True):
    df_grouped = df.groupby(['Location'])
    for name, df_subset in tqdm(df_grouped):
        paths = df_subset.Path.tolist() 
        imgs = joblib_loop(read_img, paths)
        
        afts = df_subset.After_bleach.tolist()
        cycles = df_subset.Cycle.tolist()
        locations = df_subset.Location.tolist()
        channels = df_subset.Channel.tolist()
            
        imgs_shift_crop = []
        for idx, img in enumerate(imgs):
            location = locations[idx]
            shift_dict_subset = shift_dict[location]

            # Get max shift values
            max_shift_x = shift_dict_subset["max"]["max_shift_x"]
            min_shift_x = shift_dict_subset["max"]["min_shift_x"]
            max_shift_y = shift_dict_subset["max"]["max_shift_y"]
            min_shift_y = shift_dict_subset["max"]["min_shift_y"]
            
            if afts[idx] == True:
                aft = 'af'
            else:
                aft = 'norm'
            
            if cycles[idx] == 'Cycle1' and aft == 'norm':
                # Crop image
                res_cropped = img[max_shift_y:min_shift_y-1, max_shift_x:min_shift_x-1]

            else:
                # Aplly shift
                shift_x, shift_y = (
                    shift_dict_subset[cycles[idx]][aft]["shift_x"],
                    shift_dict_subset[cycles[idx]][aft]["shift_y"],
                )
                rows, cols = img.shape
                M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
                res = cv2.warpAffine(img, M, (cols, rows))

                # Crop image
                res_cropped = res[max_shift_y:min_shift_y-1, max_shift_x:min_shift_x-1]
            imgs_shift_crop.append(res_cropped)
        Parallel(n_jobs=10)(delayed(cv2.imwrite)(j, i) for i,j in zip(imgs_shift_crop, paths))
        

In [80]:
shift_dict = get_shift_between_cycle(df[df.Location < 64])

location:   0%|          | 0/63 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

...

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

In [101]:
shift_dict

defaultdict(dict,
            {1: defaultdict(dict,
                         {'Cycle2': {'norm': {'shift_x': 9.0, 'shift_y': -8.0},
                           'af': {'shift_x': 0.0, 'shift_y': -6.0}},
                          'Cycle3': {'norm': {'shift_x': 5.0, 'shift_y': 5.0},
                           'af': {'shift_x': -4.0, 'shift_y': 0.0}},
                          'Cycle4': {'norm': {'shift_x': -5.0,
                            'shift_y': -24.0},
                           'af': {'shift_x': 8.0, 'shift_y': -2.0}},
                          'Cycle5': {'norm': {'shift_x': 22.0,
                            'shift_y': -12.0},
                           'af': {'shift_x': 12.0, 'shift_y': -30.0}},
                          'Cycle6': {'norm': {'shift_x': 11.0,
                            'shift_y': -29.0},
                           'af': {'shift_x': 26.0, 'shift_y': 2.0}},
                          'Cycle7': {'norm': {'shift_x': 30.0, 'shift_y': 1.0},
                           'af':

In [110]:
data = []
index = []
for k,v in shift_dict.items():
    row = {}
    index.append(k)
    for loc, shift in v.items():
        if loc != 'max' and loc != 'norm':
            if "norm" in shift:
                row[loc+'_norm_x'] = shift['norm']['shift_x'] 
                row[loc+'_norm_y'] = shift['norm']['shift_y'] 
            else:
                row[loc+'_norm_x'] = 0
                row[loc+'_norm_y'] = 0
            if 'af' in shift:
                row[loc+'_af_x'] = shift['af']['shift_x'] 
                row[loc+'_af_x'] = shift['af']['shift_y']
            else:
                row[loc+'_af_x'] = 0
                row[loc+'_af_x'] = 0
    data.append(row)

In [112]:
pd.DataFrame(data, index=index).to_clipboard()  

In [68]:
# get_shift_within_cycle(df)

In [71]:
shift_file = data_meta / 'shift.pkl'

if not shift_file.is_file():
    shift_dict = get_shift_between_cycle(df[df.Location < 64])
    with open(data_meta / 'shift.pkl', 'wb') as file:
        pickle.dump(shift_dict, file)
else: 
    with open(data_meta / 'shift.pkl', 'rb') as file:
        shift_dict = pickle.load(file)

location:   0%|          | 0/63 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

...

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

cycle:   0%|          | 0/10 [00:00<?, ?it/s]

In [78]:
df_combined = pd.read_csv(data_meta / 'info_combined.csv')
df_combined['After_bleach'] = df_combined['After_bleach'].astype(bool)
df_combined.to_csv(data_meta / 'info_combined.csv', index=False)
df_combined_subset = df_combined[df_combined.After_bleach == False]
df_combined_subset.head()

,Cycle,Location,Channel,Marker,Path,After_bleach
0,Cycle1,1,CH1,Hoeschst,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,False
2,Cycle1,1,CH2,p-EGFR,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,False
4,Cycle10,1,CH2,Concanavalin A,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,False
6,Cycle10,1,CH1,Hoeschst,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,False
8,Cycle10,1,CH4,PCNA,Y:\coskun-lab\Thomas\Spatial signaling\dataset...,False


In [77]:
shift_crop(df_combined_subset, shift_dict)

  0%|          | 0/63 [00:00<?, ?it/s]